This script performs the preprocessing of electricity load data for Germany. Here's a detailed breakdown of the steps:

# Preprocessing Documentation


## Data Loading and Initial Processing

In [42]:
# imports
import pandas as pd
import glob
import os

In [43]:
# Define the Paths for loading the data
interim_folder = "../data/interim"

# Load the CSV with the data for all years
df_all_years = pd.read_csv(os.path.join(interim_folder, "df_all_years.csv"), sep=';', decimal=',')

# Take a quick look at the data
df_all_years

,Datum von,Datum bis,Netzlast [MWh] Berechnete Auflösungen,Netzlast inkl. Pumpspeicher [MWh] Berechnete Auflösungen,Pumpspeicher [MWh] Berechnete Auflösungen,Residuallast [MWh] Berechnete Auflösungen
0,2021-01-01 00:00:00,01.01.2021 01:00,"44.569,25","44.957,50","388,25","40.257,00"
1,2021-01-01 01:00:00,01.01.2021 02:00,"42.806,00","43.124,50","318,50","38.882,25"
2,2021-01-01 02:00:00,01.01.2021 03:00,"41.049,75","42.862,50","1.812,75","37.545,50"
3,2021-01-01 03:00:00,01.01.2021 04:00,"40.233,75","43.908,50","3.674,75","37.144,75"
4,2021-01-01 04:00:00,01.01.2021 05:00,"40.210,50","44.539,25","4.328,75","37.451,00"
...,...,...,...,...,...,...
35055,2024-12-31 19:00:00,31.12.2024 20:00,"54.382,75","54.828,50","445,75","23.523,75"
35056,2024-12-31 20:00:00,31.12.2024 21:00,"51.189,00","51.763,00","574,00","18.822,75"
35057,2024-12-31 21:00:00,31.12.2024 22:00,"49.336,50","49.923,50","587,00","16.247,00"
35058,2024-12-31 22:00:00,31.12.2024 23:00,"48.491,25","48.769,00","277,75","16.087,00"


## Data Type Conversion

In [44]:
# Check the data types of the columns
df_all_years.dtypes

Datum von                                                   object
Datum bis                                                   object
Netzlast [MWh] Berechnete Auflösungen                       object
Netzlast inkl. Pumpspeicher [MWh] Berechnete Auflösungen    object
Pumpspeicher [MWh] Berechnete Auflösungen                   object
Residuallast [MWh] Berechnete Auflösungen                   object
dtype: object

In [45]:
# Convert 'Datum von' to datetime
df_all_years['Datum von'] = pd.to_datetime(df_all_years['Datum von'])

# Convert 'Datum bis' to datetime
df_all_years['Datum bis'] = pd.to_datetime(df_all_years['Datum bis'], dayfirst=True)

# Convert load column to float
df_all_years['Netzlast [MWh] Berechnete Auflösungen'] = (
    df_all_years['Netzlast [MWh] Berechnete Auflösungen']
    .str.replace('.', '', regex=False)  # remove thousand separator
    .str.replace(',', '.', regex=False) # replace decimal comma with dot
    .astype(float)
)

# Quick check of datatypes
print(df_all_years.dtypes)

Datum von                                                   datetime64[ns]
Datum bis                                                   datetime64[ns]
Netzlast [MWh] Berechnete Auflösungen                              float64
Netzlast inkl. Pumpspeicher [MWh] Berechnete Auflösungen            object
Pumpspeicher [MWh] Berechnete Auflösungen                           object
Residuallast [MWh] Berechnete Auflösungen                           object
dtype: object


## Data Completeness Check

- Verifies hourly coverage for each year in the dataset
- Identifies missing hours

In [46]:
# Ensure 'Datum von' is datetime
df_all_years['Datum von'] = pd.to_datetime(df_all_years['Datum von'])

# Check hourly coverage per year without changing the index
years = df_all_years['Datum von'].dt.year.unique()

for year in years:
    # Full hourly range for this year
    full_range = pd.date_range(start=f'{year}-01-01 00:00', end=f'{year}-12-31 23:00', freq='h')
    
    # Select data for this year
    df_year = df_all_years[df_all_years['Datum von'].dt.year == year]
    
    # Find missing timestamps
    missing_hours = full_range.difference(df_year['Datum von'])
    
    if len(missing_hours) == 0:
        print(f"{year}: All hours present")
    else:
        print(f"{year}: Missing {len(missing_hours)} hours")
        print(missing_hours)


2021: Missing 1 hours
DatetimeIndex(['2021-03-28 02:00:00'], dtype='datetime64[ns]', freq='h')
2022: Missing 1 hours
DatetimeIndex(['2022-03-27 02:00:00'], dtype='datetime64[ns]', freq='h')
2023: Missing 1 hours
DatetimeIndex(['2023-03-26 02:00:00'], dtype='datetime64[ns]', freq='h')
2024: Missing 1 hours
DatetimeIndex(['2024-03-31 02:00:00'], dtype='datetime64[ns]', freq='h')


The missing hours in the dataset correspond to the daylight saving time (DST) transitions in Germany, when clocks are set forward by one hour in spring. These hours never actually occur, so filling them with NaN would be incorrect. It is expected and normal that these timestamps are absent, and their absence does not indicate missing data.


## Data Cleaning

- Renames columns to English
- Removes unnecesarry columns
- Results in a clean dataset with standardized column names

In [47]:
print(df_all_years.columns.tolist())


['Datum von', 'Datum bis', 'Netzlast [MWh] Berechnete Auflösungen', 'Netzlast inkl. Pumpspeicher [MWh] Berechnete Auflösungen', 'Pumpspeicher [MWh] Berechnete Auflösungen', 'Residuallast [MWh] Berechnete Auflösungen']


In [48]:
# Rename and translate columns
df_all_years = df_all_years.rename(columns={
    'Datum von': 'timestamp',      
    'Netzlast [MWh] Berechnete Auflösungen': 'load_MWh'
})

# Drop 'Datum bis' column
df_all_years = df_all_years.loc[:, ['timestamp', 'load_MWh']]

# Quick check
df_all_years

,timestamp,load_MWh
0,2021-01-01 00:00:00,44569.25
1,2021-01-01 01:00:00,42806.00
2,2021-01-01 02:00:00,41049.75
3,2021-01-01 03:00:00,40233.75
4,2021-01-01 04:00:00,40210.50
...,...,...
35055,2024-12-31 19:00:00,54382.75
35056,2024-12-31 20:00:00,51189.00
35057,2024-12-31 21:00:00,49336.50
35058,2024-12-31 22:00:00,48491.25


In [49]:
# Check for missing values
missing_counts = df_all_years.isna().sum()
print(missing_counts)

timestamp    0
load_MWh     0
dtype: int64


In [50]:
# Check year-to-year transitions
# This ensures that the last hours of each year correctly connect to the first hours of the next year.
# It helps verify that there are no gaps or overlaps at the year boundaries.
# Check the last 3 entries of each year
years = df_all_years['timestamp'].dt.year.unique()

for year in years:
    df_year = df_all_years[df_all_years['timestamp'].dt.year == year]
    print(f"\nYear {year}:")
    print("Last 3 hours:")
    print(df_year.tail(3))
    print("First 3 hours of next year:")
    df_next = df_all_years[df_all_years['timestamp'].dt.year == year+1]
    print(df_next.head(3))


Year 2021:
Last 3 hours:
               timestamp  load_MWh
8756 2021-12-31 21:00:00  48751.75
8757 2021-12-31 22:00:00  47461.50
8758 2021-12-31 23:00:00  45616.75
First 3 hours of next year:
               timestamp  load_MWh
8759 2022-01-01 00:00:00  43915.50
8760 2022-01-01 01:00:00  41535.75
8761 2022-01-01 02:00:00  40480.75

Year 2022:
Last 3 hours:
                timestamp  load_MWh
17515 2022-12-31 21:00:00   44002.0
17516 2022-12-31 22:00:00   42489.5
17517 2022-12-31 23:00:00   40646.5
First 3 hours of next year:
                timestamp  load_MWh
17518 2023-01-01 00:00:00  38346.00
17519 2023-01-01 01:00:00  37777.25
17520 2023-01-01 02:00:00  36939.75

Year 2023:
Last 3 hours:
                timestamp  load_MWh
26274 2023-12-31 21:00:00  45158.75
26275 2023-12-31 22:00:00  44088.50
26276 2023-12-31 23:00:00  42140.00
First 3 hours of next year:
                timestamp  load_MWh
26277 2024-01-01 00:00:00  40170.25
26278 2024-01-01 01:00:00  38818.25
26279 2024-01-01 0

In [51]:
# Define output file path
output_path = os.path.join(interim_folder, "cleaned_df.csv")

# Save the DataFrame
df_all_years.to_csv(output_path, sep=';', decimal=',', index=False)

print(f"cleaned_df saved to {output_path}")

cleaned_df saved to ../data/interim/cleaned_df.csv
